In [49]:
!pip install keras-tuner 


In [50]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 

In [51]:
print(tf.__version__)

2.3.0


In [52]:
fashion_mnist = keras.datasets.fashion_mnist

In [55]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [54]:
train_images = train_images/255.0
test_images = test_images/255.0

In [56]:
train_images[0].shape

(28, 28)

In [57]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [58]:
def build_model(hp):
  model = keras.Sequential([keras.layers.Conv2D(filters= hp.Int('conv_1_filter',
                                                                min_value=32,
                                                                max_value=128,
                                                                step=16),
                                             kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                                             activation='relu',
                                             input_shape=(28,28,1)
                                             ),
          keras.layers.Conv2D(filters=hp.Int('conv_2_filter', min_value=32,max_value=128,step=16),
                              kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                              activation='relu'
                              ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units=hp.Int('dense_1_units', min_value=32, max_value=128,step=16),
              activation= 'relu'
          ),
          keras.layers.Dense(10,activation='softmax')])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [59]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [61]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='my_dir', project_name="Mnist Fashion")

In [64]:
tuner_search.search(train_images,train_labels,epochs= 3, validation_split=0.1)
tuner_search.results_summary()

INFO:tensorflow:Oracle triggered exit
Results summary
Results in my_dir/Mnist Fashion
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.8891666531562805
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 5
conv_2_filter: 80
conv_2_kernel: 5
dense_1_units: 112
learning_rate: 0.001
Score: 0.8765000104904175
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 128
conv_2_kernel: 3
dense_1_units: 64
learning_rate: 0.01
Score: 0.8411666750907898
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 3
conv_2_filter: 96
conv_2_kernel: 3
dense_1_units: 64
learning_rate: 0.01
Score: 0.7991666793823242
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 80
conv_2_kernel: 3
dense_1_units: 80
learning_rate: 0.01
Score: 0.7985000014305115


In [45]:
model = tuner_search.get_best_models(num_models=1)[0]


In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        102464    
_________________________________________________________________
flatten (Flatten)            (None, 25600)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2867312   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,972,570
Trainable params: 2,972,570
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1542 - accuracy: 0.9415 - val_loss: 0.2656 - val_accuracy: 0.9068
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1184 - accuracy: 0.9570 - val_loss: 0.2794 - val_accuracy: 0.9102
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0883 - accuracy: 0.9669 - val_loss: 0.3090 - val_accuracy: 0.9118
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0669 - accuracy: 0.9747 - val_loss: 0.3366 - val_accuracy: 0.9150
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0502 - accuracy: 0.9817 - val_loss: 0.4367 - val_accuracy: 0.9078
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0415 - accuracy: 0.9841 - val_loss: 0.4114 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0344 - accuracy: 0.9876 - val_loss: 0.4753 - val_accuracy